In [ ]:
# Mise à jour de PIP
!python.exe -m pip install --upgrade pip -q

# PyTorch avec une syntaxe plus robuste
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121 -q
print("✓ PyTorch Suite installée")

# Reconnaissance faciale & Vision (Installation groupée)
!pip install facenet-pytorch opencv-python opencv-contrib-python Pillow -q
print("✓ Vision (FaceNet, OpenCV, Pillow) installée")

# (Attention : NumPy 2.x par défaut ici)
!pip install "numpy>=2.0" scipy pandas scikit-learn -q
print("✓ NumPy 2.x+, SciPy, Pandas, scikit-learn installés")

# 4. Visualisation
!pip install matplotlib seaborn -q
print("✓ matplotlib, seaborn installés")

# 5. Utilitaires & Environnement
!pip install tqdm ipython jupyter ipykernel joblib psutil python-dateutil -q
print("✓ Utilitaires installés")


Installation de toutes les dépendances...

✓ PyTorch Suite installée
✓ Vision (FaceNet, OpenCV, Pillow) installée
✓ NumPy 2.x+, SciPy, Pandas, scikit-learn installés
✓ matplotlib, seaborn installés
✓ Utilitaires installés
 ======= TOUTES LES DÉPENDANCES SONT PRÊTES !  =======


In [ ]:
import cv2
import os
import numpy as np
import time

def capture_150_faces(person_name, output_dir="dataset"):
    """
    Capture automatique de 150 images de visage
    """
    # Créer dossier
    person_dir = os.path.join(output_dir, person_name)
    os.makedirs(person_dir, exist_ok=True)
    
    # Détecteur amélioré
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Webcam
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print(f"\nCapture de 150 images pour {person_name}...")
    
    count = 0
    last_capture = 0
    capture_delay = 0.2  # 200ms entre captures
    
    while count < 150:
        ret, frame = cap.read()
        if not ret:
            break
        
        current_time = time.time()
        
        # Détection visage
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(100, 100))
        
        display = frame.copy()
        
        if len(faces) > 0:
            # Prendre le plus grand visage
            faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
            x, y, w, h = faces[0]
            
            # Dessiner rectangle
            cv2.rectangle(display, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Capturer si temps écoulé
            if current_time - last_capture > capture_delay:
                # Vérifier netteté
                face_roi = gray[y:y+h, x:x+w]
                sharpness = cv2.Laplacian(face_roi, cv2.CV_64F).var()
                
                if sharpness > 15:  # Seuil de netteté
                    face = frame[y:y+h, x:x+w]
                    face_resized = cv2.resize(face, (160, 160))
                    
                    # Sauvegarder
                    filename = f"{count:03d}.jpg"
                    cv2.imwrite(os.path.join(person_dir, filename), face_resized)
                    
                    count += 1
                    last_capture = current_time
                    
                    # Augmentation simple
                    if count % 3 == 0:
                        face_flipped = cv2.flip(face_resized, 1)
                        cv2.imwrite(os.path.join(person_dir, f"{count:03d}_f.jpg"), face_flipped)
        
        # Affichage info
        cv2.putText(display, f"Images: {count}/150", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        # Barre progression
        progress = int((count / 150) * 300)
        cv2.rectangle(display, (10, 50), (10 + progress, 60), (0, 255, 0), -1)
        
        cv2.imshow('Capture Automatique', display)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"\n {count} images capturées dans: {person_dir}")
    return count



In [4]:
if __name__ == "__main__":
    name = input("Nom de la personne: ").strip()
    if name:
        capture_150_faces(name)


Capture de 150 images pour mustapha...

 150 images capturées dans: dataset\mustapha
